# 1. Setup

In [3]:
# some imports
import sys
import os
import MCMC_Noddy as mcmc     

# give correct permissions to the executable
folder = os.getcwd()
noddyEXE = folder+'/noddy_linux.exe'
strV = 'chmod 777 '+noddyEXE
os.system(strV)

0

# 2. Choose hyper parameters

In [4]:
HyperParameters = {}

HyperParameters['ScenarioNum'] = 18
HyperParameters['SimulationShiftType'] = 'Median Datum Shift'
HyperParameters['DatNormMethod'] = 'MedianInitialRounds'
HyperParameters['ExplorationRate'] = 'LinearErrorBased'
HyperParameters['ErrorType'] = 'Global'
HyperParameters['ErrorNorm'] = 'L1'
HyperParameters['ExplorationStage'] = 'Explore'
HyperParameters['AcceptProbType'] = 'Track Acceptance'
HyperParameters['cubesize'] = 150
HyperParameters['AcceptanceGoal'] = 0.2
HyperParameters['ConstNormFactor'] = 0.01
HyperParameters['GlobalMoveEachDir'] = 500
HyperParameters['XYZ_Axes_StepStd'] = 100
HyperParameters['Dip_StepStd'] = 5
HyperParameters['Slip_StepStd'] = 50
HyperParameters['DipDirection_StepStd'] = 6
HyperParameters['SlipParam'] = 0.1
HyperParameters['SteppingSizeMult'] = 1/0.9
HyperParameters['MaxFaultMarkerError'] = 525
HyperParameters['AmplitudeRatioChange'] = 0.1
HyperParameters['AzimuthMoveEachDirection'] = 6
HyperParameters['AxisRatioChange'] = 0.1
HyperParameters['DipMoveEachDirection'] = 32
HyperParameters['nExploreRuns'] = 50
HyperParameters['MO_WeightingMethod'] = 'Proportions'
HyperParameters['MCMC_SwitchWeightFreq'] = 20
HyperParameters['LocalWeightsMode'] = 'na'

HyperParameters['thread_num'] = 15        
HyperParameters['Toy']=False
HyperParameters['verbose']=True
HyperParameters['nruns']=200
HyperParameters['BaseFolder']='Combo_Scratch'
HyperParameters['GeneralPerturbStrategy']='OnlyGlobal'
HyperParameters['ControlPointMovementUnits'] = 'Absolute value'
HyperParameters['errCalcMethodFaultMarker']= 'Distance'
HyperParameters['DataTypes'] = ['Grav', 'Mag', 'GT', 'FaultMarkers']
HyperParameters['JoinType']='LINES'
HyperParameters['xy_origin']=[316448, 4379166, -2700]
HyperParameters['xy_extent'] = [8850, 9000,3900]
HyperParameters['DatNormCoef'] = {'Grav': 2.4, 'Tracer': 1.0, 
                        'FaultMarkers': 500, 'GT': 315, 'Mag':300}
HyperParameters['verbose']=True    
HyperParameters['graniteIdx'] = 4
HyperParameters['Windows'] = False
HyperParameters['jupyter'] = True

HyperParameters['OutputImageFreq'] = 15        
HyperParameters['OptimMethod']='MCMC'

# 3. Run and view results every x times (x=output image frequency)

In [5]:
if(HyperParameters['OptimMethod']=='MCMC'):
    mcmc.MCMC_Noddy(HyperParameters)

Finished calculating geology and topology
Finished calculating gravity and magnetics
There might be an issue


KeyError: 'xEdge'

# 4. Find the model with least error

In [9]:
#find 

'chmod 777 /home/jovyan/code/noddy.exe'

# 5. Plot in 3D